In [40]:
import pandas as pd
import numpy as np

# Read in data
codon_usage = pd.read_csv('codon_usage.csv')
codon_usage.head()
codon_usage['DNAtype'].value_counts()
codon_usage['Kingdom'].value_counts()
codon_usage.shape # (13028, 69)
codon_usage['SpeciesID'].nunique() # 12368
codon_usage.dtypes

/var/folders/_q/_22r26l55_n_rj5snfxxfpn80000gn/T/ipykernel_75964/1366451953.py:5: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  codon_usage = pd.read_csv('codon_usage.csv')


Kingdom         object
DNAtype          int64
SpeciesID        int64
Ncodons          int64
SpeciesName     object
                ...   
GAA            float64
GAG            float64
UAA            float64
UAG            float64
UGA            float64
Length: 69, dtype: object

In [41]:
# drop 'SpeciesID', 'SpeciesName'
codon_usage = codon_usage.drop(columns=['SpeciesID', 'SpeciesName'])

In [42]:
# Get one hot encoding of columns 'DNAtype'
one_hot = pd.get_dummies(codon_usage['DNAtype'])
codon_usage = codon_usage.drop('DNAtype', axis=1)
codon_usage = codon_usage.join(one_hot)
codon_usage.head() # (13028, 70)

,Kingdom,Ncodons,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,...,1,2,3,4,5,6,7,9,11,12
0,vrl,1995,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,...,False,False,False,False,False,False,False,False,False,False
1,vrl,1474,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,...,False,False,False,False,False,False,False,False,False,False
2,vrl,4862,0.01974,0.0218,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,...,False,False,False,False,False,False,False,False,False,False
3,vrl,1915,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,...,False,False,False,False,False,False,False,False,False,False
4,vrl,22831,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,...,False,False,False,False,False,False,False,False,False,False


In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(codon_usage.drop('Kingdom', axis=1), codon_usage['Kingdom'], test_size=0.20, random_state=101)

In [49]:
# standardize 'Ncodons'
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(X_train['Ncodons'].values.reshape(-1,1))
X_train['Ncodons'] = scaler.transform(X_train['Ncodons'].values.reshape(-1,1))
X_test['Ncodons'] = scaler.transform(X_test['Ncodons'].values.reshape(-1,1))
X_test

,Ncodons,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,...,1,2,3,4,5,6,7,9,11,12
4122,-0.042989,0.00077,0.0282,0.00000,0.00463,0.00309,0.05639,0.00154,0.04017,0.00116,...,False,False,False,False,False,False,False,False,False,False
292,2.342783,0.02913,0.01961,0.02141,0.03258,0.01947,0.01233,0.01588,0.01874,0.02287,...,False,False,False,False,False,False,False,False,False,False
5118,-0.212294,0.02593,0.0122,0.04729,0.01602,0.01907,0.00458,0.01297,0.00458,0.04882,...,False,False,False,False,False,False,False,False,False,False
4887,62.865934,0.03015,0.01558,0.01755,0.02563,0.02817,0.01125,0.01038,0.00608,0.04322,...,False,False,False,False,False,False,False,False,False,False
10537,-0.149765,0.02019,0.05048,0.01739,0.00841,0.03758,0.02580,0.03702,0.02299,0.04262,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5772,1.145857,0.02396,0.01609,0.01790,0.02560,0.01410,0.01678,0.01168,0.03001,0.02793,...,False,False,False,False,False,False,False,False,False,False
5806,-0.195469,0.05702,0.01113,0.06885,0.01252,0.00904,0.00139,0.00974,0.00209,0.06676,...,False,False,False,False,False,False,False,False,False,False
5742,3.405908,0.0313,0.01279,0.04283,0.00880,0.02054,0.00555,0.01411,0.00779,0.04954,...,False,False,False,False,False,False,False,False,False,False
2731,-0.144863,0.01813,0.01264,0.01484,0.01099,0.01978,0.02088,0.00934,0.02253,0.01538,...,False,False,False,False,False,False,False,False,False,False


In [51]:
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('Y_train.csv', index=False)
y_test.to_csv('Y_test.csv', index=False)